In [150]:
import numpy as np
import pandas as pd
import datetime as dt

import plotly
import plotly.express as px
import plotly.graph_objects as go

import chart_studio.tools as tls

In [4]:
# get the path to our data
path = r"F:\GitHub\Projects\MyLifeinData\data"
# read in our data
smt = pd.read_csv(path + r"\timeslots.csv")

In [31]:
# convert our string object into a datetime object
smt["Day"] = [dt.datetime.strptime(str(date), "%Y%m%d") for date in smt["Day"]]

In [48]:
smt

,Day,Timestamp UTC ms,Time,Place,Room,Activity,Activity Category,Duration ms,Steps,Move,Device
0,2018-12-31,1546266604128,Mon Dec 31 09:30:04 EST 2018,Parent's House,NaN,Waking up,Resting,7272585,328,WALK,NaN
1,2018-12-31,1546273876713,Mon Dec 31 11:31:16 EST 2018,NaN,NaN,car,Transport,1465125,38,VEHICLE,NaN
2,2018-12-31,1546275341838,Mon Dec 31 11:55:41 EST 2018,Panera Bread,NaN,eat out,Social,1874949,442,WALK,NaN
3,2018-12-31,1546277216787,Mon Dec 31 12:26:56 EST 2018,Michaels,NaN,walk,Transport,1808805,608,WALK,NaN
4,2018-12-31,1546279025592,Mon Dec 31 12:57:05 EST 2018,NaN,NaN,groceries,Chores,3224212,1197,WALK,NaN
...,...,...,...,...,...,...,...,...,...,...,...
5071,2019-11-22,1574432968962,Fri Nov 22 09:29:28 EST 2019,NaN,NaN,call: 19292056099,Professional Life,1238649,0,VEHICLE,LG G4
5072,2019-11-22,1574434207611,Fri Nov 22 09:50:07 EST 2019,NaN,NaN,car,Transport,500351,0,VEHICLE,NaN
5073,2019-11-22,1574434707962,Fri Nov 22 09:58:27 EST 2019,4431 manor village way,NaN,family time,Personal Life,1908358,0,NOMOVE,NaN
5074,2019-11-22,1574436616320,Fri Nov 22 10:30:16 EST 2019,4431 manor village way,NaN,video games,Art & Entertainment,5410440,0,NOMOVE,NaN


In [159]:
# get the YYYY, MM, and DD columns from our datetime object
years = pd.DataFrame([date.year for date in (smt["Day"])], columns=["Year"])
months = pd.DataFrame([date.month for date in (smt["Day"])], columns=["Month"])
days = pd.DataFrame([date.day for date in (smt["Day"])], columns=["Day"])

# keep only the columns that we really need
grouped_df = pd.concat([smt["Activity Category"], smt["Duration ms"]/360000, years, months, days], axis=1)

# sum each of our groups to prep our data for plotting
grouped_df = grouped_df.groupby(["Year", "Month", "Activity Category"]).sum().reset_index()
grouped_df = grouped_df.rename(columns={"Duration ms" : "Duration (Hours)"})

# split our data to not be past May, 2019
only_may = [i for i, x in enumerate(grouped_df["Month"]) if x < 5]
grouped_df = grouped_df.iloc[only_may]

# remove unnecessary categories
less_groups = [i for i, x in enumerate(grouped_df["Activity Category"]) if x not in ["Chores", "Health & Hygiene",
                                                                                "Resting", "Social", "Transport",
                                                                                "Sport & Fitness"]]
grouped_df = grouped_df.iloc[less_groups]


# get our x-axis correct
grouped_df["Year-Month"] = grouped_df["Year"].map(str) + "-" + grouped_df["Month"].map(str)

In [163]:
fig = px.bar(grouped_df, x="Month", y="Duration (Hours)", color='Activity Category', barmode='group')

fig.update_layout(legend=dict(x=0, y=1.2))

fig.show()

In [164]:
need = '<script src="https://cdn.plot.ly/plotly-latest.min.js"></script> '
out = plotly.offline.plot(fig, include_plotlyjs=False, output_type='div')
print(need + out)

<script src="https://cdn.plot.ly/plotly-latest.min.js"></script> <div>
        
        
            <div id="757024db-2c93-4587-a403-da96e658da11" class="plotly-graph-div" style="height:600px; width:100%;"></div>
            <script type="text/javascript">
                
                    window.PLOTLYENV=window.PLOTLYENV || {};
                    
                if (document.getElementById("757024db-2c93-4587-a403-da96e658da11")) {
                    Plotly.newPlot(
                        '757024db-2c93-4587-a403-da96e658da11',
                        [{"alignmentgroup": "True", "hoverlabel": {"namelength": 0}, "hovertemplate": "Activity Category=Art & Entertainment<br>Month=%{x}<br>Duration (Hours)=%{y}", "legendgroup": "Activity Category=Art & Entertainment", "marker": {"color": "#636efa"}, "name": "Activity Category=Art & Entertainment", "offsetgroup": "Activity Category=Art & Entertainment", "orientation": "v", "showlegend": true, "textposition": "auto", "type": "bar", "x